In [57]:
#Dependancies
import pandas as pd
from sqlalchemy import *
from sqlalchemy.schema import CreateSchema
import pymysql
from config import username,pwd,host
import pymongo

In [27]:
#importing csv file
csvfile="data/Census_CSV_data.csv"
df=pd.read_csv(csvfile)

/anaconda3/envs/PythonEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
df.head()

,CENSUS_YEAR,GEO_CODE (POR),GEO_LEVEL,GEO_NAME,GNR,GNR_LF,DATA_QUALITY_FLAG,ALT_GEO_CODE,DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247),Member ID: Profile of Federal Electoral Districts (2013 Representation Order) (2247),Notes: Profile of Federal Electoral Districts (2013 Representation Order) (2247),Dim: Sex (3): Member ID: [1]: Total - Sex,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
0,2016,1,0,Canada,4.0,5.1,20000,1,"Population, 2016",1,1.0,3.51517e+07,...,...
1,2016,1,0,Canada,4.0,5.1,20000,1,"Population, 2011",2,2.0,3.34767e+07,...,...
2,2016,1,0,Canada,4.0,5.1,20000,1,"Population percentage change, 2011 to 2016",3,NaN,5,...,...
3,2016,1,0,Canada,4.0,5.1,20000,1,Total private dwellings,4,3.0,1.54124e+07,...,...
4,2016,1,0,Canada,4.0,5.1,20000,1,Private dwellings occupied by usual residents,5,4.0,1.40721e+07,...,...


In [29]:
#rows*columns
df.shape

(790944, 14)

In [30]:
#column names
df.columns

Index(['CENSUS_YEAR', 'GEO_CODE (POR)', 'GEO_LEVEL', 'GEO_NAME', 'GNR',
       'GNR_LF', 'DATA_QUALITY_FLAG', 'ALT_GEO_CODE',
       'DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)',
       'Member ID: Profile of Federal Electoral Districts (2013 Representation Order) (2247)',
       'Notes: Profile of Federal Electoral Districts (2013 Representation Order) (2247)',
       'Dim: Sex (3): Member ID: [1]: Total - Sex',
       'Dim: Sex (3): Member ID: [2]: Male',
       'Dim: Sex (3): Member ID: [3]: Female'],
      dtype='object')

In [31]:
#Selecting required columns
df=df[['CENSUS_YEAR','GEO_LEVEL','GEO_NAME','DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)','Dim: Sex (3): Member ID: [1]: Total - Sex','Dim: Sex (3): Member ID: [2]: Male','Dim: Sex (3): Member ID: [3]: Female']]
      

In [32]:
df.head()

,CENSUS_YEAR,GEO_LEVEL,GEO_NAME,DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247),Dim: Sex (3): Member ID: [1]: Total - Sex,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
0,2016,0,Canada,"Population, 2016",3.51517e+07,...,...
1,2016,0,Canada,"Population, 2011",3.34767e+07,...,...
2,2016,0,Canada,"Population percentage change, 2011 to 2016",5,...,...
3,2016,0,Canada,Total private dwellings,1.54124e+07,...,...
4,2016,0,Canada,Private dwellings occupied by usual residents,1.40721e+07,...,...


In [33]:
df.shape

(790944, 7)

In [34]:
# Renaming columns
df=df.rename(columns = {'DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)': "Metric",'Dim: Sex (3): Member ID: [1]: Total - Sex':"Total",'Dim: Sex (3): Member ID: [2]: Male': 'Male','Dim: Sex (3): Member ID: [3]: Female': 'Female'})
df=df.rename(columns = {'CENSUS_YEAR':'Census_year','GEO_LEVEL':'Geo_level','GEO_NAME':'Geo_name'})
df.head()

,Census_year,Geo_level,Geo_name,Metric,Total,Male,Female
0,2016,0,Canada,"Population, 2016",3.51517e+07,...,...
1,2016,0,Canada,"Population, 2011",3.34767e+07,...,...
2,2016,0,Canada,"Population percentage change, 2011 to 2016",5,...,...
3,2016,0,Canada,Total private dwellings,1.54124e+07,...,...
4,2016,0,Canada,Private dwellings occupied by usual residents,1.40721e+07,...,...


In [35]:
df.shape

(790944, 7)

In [36]:
df_population_2016=df.loc[df["Metric"]=="Population, 2016", :]
df_population_2016.insert(0,'Id',range(1,1+len(df_population_2016)))
df_population_2016.head()

,Id,Census_year,Geo_level,Geo_name,Metric,Total,Male,Female
0,1,2016,0,Canada,"Population, 2016",3.51517e+07,...,...
2247,2,2016,1,Newfoundland and Labrador,"Population, 2016",519716,...,...
4494,3,2016,2,Avalon,"Population, 2016",86494,...,...
6741,4,2016,2,Bonavista--Burin--Trinity,"Population, 2016",74116,...,...
8988,5,2016,2,Coast of Bays--Central--Notre Dame,"Population, 2016",77680,...,...


In [37]:
df_population_2016.shape

(352, 8)

In [51]:
prov_2016=df_population_2016.loc[df_population_2016['Geo_level']==1,:]
prov_2016

,Id,Census_year,Geo_level,Geo_name,Metric,Total,Male,Female
2247,2,2016,1,Newfoundland and Labrador,"Population, 2016",519716,...,...
20223,10,2016,1,Prince Edward Island,"Population, 2016",142907,...,...
31458,15,2016,1,Nova Scotia,"Population, 2016",923598,...,...
58422,27,2016,1,New Brunswick,"Population, 2016",747101,...,...
83139,38,2016,1,Quebec,"Population, 2016",8.16436e+06,...,...
260652,117,2016,1,Ontario,"Population, 2016",1.34485e+07,...,...
534786,239,2016,1,Manitoba,"Population, 2016",1.27836e+06,...,...
568491,254,2016,1,Saskatchewan,"Population, 2016",1.09835e+06,...,...
602196,269,2016,1,Alberta,"Population, 2016",4.06718e+06,...,...
680841,304,2016,1,British Columbia,"Population, 2016",4.64806e+06,...,...


In [52]:
csvfile="data/Ridings.csv"
df_ridings=pd.read_csv(csvfile)
df_ridings

,Geo_name,Ridings
0,Newfoundland and Labrador,7
1,Prince Edward Island,4
2,Nova Scotia,11
3,New Brunswick,10
4,Ontario,121
5,Quebec,78
6,Saskatchewan,14
7,Manitoba,14
8,Alberta,34
9,British Columbia,42


In [64]:
prov_2016=prov_2016.merge(df_ridings,'left',on='Geo_name')
prov_2016



,Id,Census_year,Geo_level,Geo_name,Metric,Total,Male,Female,Ridings_x,Ridings_y
0,2,2016,1,Newfoundland and Labrador,"Population, 2016",519716,...,...,7,7
1,10,2016,1,Prince Edward Island,"Population, 2016",142907,...,...,4,4
2,15,2016,1,Nova Scotia,"Population, 2016",923598,...,...,11,11
3,27,2016,1,New Brunswick,"Population, 2016",747101,...,...,10,10
4,38,2016,1,Quebec,"Population, 2016",8.16436e+06,...,...,78,78
5,117,2016,1,Ontario,"Population, 2016",1.34485e+07,...,...,121,121
6,239,2016,1,Manitoba,"Population, 2016",1.27836e+06,...,...,14,14
7,254,2016,1,Saskatchewan,"Population, 2016",1.09835e+06,...,...,14,14
8,269,2016,1,Alberta,"Population, 2016",4.06718e+06,...,...,34,34
9,304,2016,1,British Columbia,"Population, 2016",4.64806e+06,...,...,42,42


In [55]:
prov_2016.to_csv('data/prov_2016.csv')

In [65]:
csvfile="data/prov_2016.csv"
df_prov=pd.read_csv(csvfile)

df_prov

,Unnamed: 0,Id,Census_year,Geo_level,Geo_name,Metric,Total,Male,Female,Ridings
0,0,2,2016,1,Newfoundland and Labrador,"Population, 2016",519716.0,...,...,7
1,1,10,2016,1,Prince Edward Island,"Population, 2016",142907.0,...,...,4
2,2,15,2016,1,Nova Scotia,"Population, 2016",923598.0,...,...,11
3,3,27,2016,1,New Brunswick,"Population, 2016",747101.0,...,...,10
4,4,38,2016,1,Quebec,"Population, 2016",8164361.0,...,...,78
5,5,117,2016,1,Ontario,"Population, 2016",13448494.0,...,...,121
6,6,239,2016,1,Manitoba,"Population, 2016",1278365.0,...,...,14
7,7,254,2016,1,Saskatchewan,"Population, 2016",1098352.0,...,...,14
8,8,269,2016,1,Alberta,"Population, 2016",4067175.0,...,...,34
9,9,304,2016,1,British Columbia,"Population, 2016",4648055.0,...,...,42


In [66]:
data = {
        "geo_name": df_prov['Geo_name'].values.tolist(),
        "population": df_prov['Total'].values.tolist(),
        "ridings": df_prov['Ridings'].values.tolist(),
    }

data

{'geo_name': ['Newfoundland and Labrador',
  'Prince Edward Island',
  'Nova Scotia',
  'New Brunswick',
  'Quebec',
  'Ontario',
  'Manitoba',
  'Saskatchewan',
  'Alberta',
  'British Columbia',
  'Yukon',
  'Northwest Territories',
  'Nunavut'],
 'population': [519716.0,
  142907.0,
  923598.0,
  747101.0,
  8164361.0,
  13448494.0,
  1278365.0,
  1098352.0,
  4067175.0,
  4648055.0,
  35874.0,
  41786.0,
  35944.0],
 'ridings': [7, 4, 11, 10, 78, 121, 14, 14, 34, 42, 1, 1, 1]}

In [14]:
# #database connection
# metadata = MetaData()
# engine = create_engine(f'mysql+pymysql://{username}:{pwd}@{host}:3306')
# # engine.execute(CreateSchema('ValueofVote'))

In [15]:
# engine.execute("USE ValueofVote")
# metadata = MetaData()

In [16]:
# #create table
# Population_2016 = Table('population_2016', metadata,
#     Column('Id', Integer, primary_key=True),
#     Column('Census_year', Integer, nullable=False),
#     Column('Geo_level', Integer, nullable=False),
#     Column('Geo_Name', String(100), nullable=False),
#     Column('Metric', String(100), nullable=False),
#     Column('Total', Integer, nullable=True),
#     Column('Male', Integer, nullable=True),
#     Column('Female', Integer, nullable=True)
# )

In [17]:
# df_population_2016.to_sql(name='population_2016', con=engine, if_exists='replace', index=False)

In [18]:
prov_json="templates/data/test_prov.geojson"
# prov=pd.read_json(prov_json)
prov_json_v2="templates/data/test_prov_v2.geojson"

import json

with open(prov_json, 'r') as pj:
    data = json.load(pj)

features=data['features']

In [19]:
for feature in features:
    feature ['properties']['Population']=(prov_2016.loc[prov_2016['Geo_name']==feature ['properties']['PRENAME'],'Total']).values[0]

features[1] ['properties']['Population']

8164361.0

In [20]:
 with open(prov_json_v2, 'w') as pj2:
        json.dump(data, pj2)

In [21]:
with open(prov_json_v2, 'r') as pj:
    prov_data = json.load(pj)
    
dict_prov={
    "prov_data":prov_data
}

In [25]:
# Setup connection to mongodb
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)


# # Select database and collection to use
# db = client.ValofVote_db

# # # Drops collection if available to remove duplicates
# db.prov_geojson.drop()
    
# # #Insert prov geojson into the database
# db.prov_geojson.insert_one(dict_prov)


DocumentTooLarge: BSON document too large (111985985 bytes) - the connected server supports BSON document sizes up to 16793598 bytes.